## Loading data

In [49]:
import re
import pandas as pd

# Prepare data
path_pkl = os.path.join("data", "test_df.pkl")
data = pd.read_pickle(path_pkl) 

print(len(data))

data = data[data["language"] == "uk"]

print(len(data))

data = data[data["confidence"] > 0.9]

print(len(data))

# make text column lower case
data.document = data['document'].map(lambda document: document.lower() if isinstance(document,str) else document)

# make lists
dates = data.date.to_list()
docs = data.document.to_list()
subreddits = data.sub_reddit.to_list()
language = data.language.to_list()
type = data.type.to_list()

421537
846
562


In [50]:
data

,document,sub_reddit,date,type,language,confidence
6,11-річна українська гімнастка катя дяченко заг...,ukraina,2022-03-23,submission,uk,0.994467
17,"ізюм, харківська область (120 км від харкова)....",ukraina,2022-03-23,submission,uk,0.987032
23,український дайджест - огляд подій за 23 березня,ukraina,2022-03-23,submission,uk,0.993930
49,"музична пауза від surface tension. ""ми вас всі...",ukraina,2022-03-23,submission,uk,0.939258
51,у рашистських загарбників – розрив шаблону від...,ukraina,2022-03-23,submission,uk,0.997888
...,...,...,...,...,...,...
449221,"мої близькі подруги з родиною недалеко, два дн...",ukraina,2022-03-07,comment,uk,0.979403
449277,до нашого відділу майєна приїжджає небагато лю...,ukraina,2022-03-07,comment,uk,0.959939
449281,#слава україні!,ukraina,2022-03-07,comment,uk,0.991116
449335,"оце мене й лякає, ще не висохла кров вбитих ро...",ukraina,2022-03-07,comment,uk,0.983650


### Checking if it works

In [51]:
pd.DataFrame.head(data)

for i in data["document"]:
    print (i)

11-річна українська гімнастка катя дяченко загинула внаслідок бомбардування маріуполя роzійськими фашистами
ізюм, харківська область (120 км від харкова). очами місцевого юнака зі смартфоном
український дайджест - огляд подій за 23 березня
музична пауза від surface tension. "ми вас всіх уб'ємо".
у рашистських загарбників – розрив шаблону від якості життя в україні
в україні оголосили першу підозру російському військовому за зґвалтування – генпрокурорка венедіктова
водіям легкових авто на період воєнного стану дозволено керувати вантажівками
використання ворогом геоміток/enemy use of geotags
маріуполь. немає москаля, а є лише сира земля!
доброго ранку, україно! 648 годин опору, дякую україна. маленька амелія співає гімн україни в лодзі, польща
арестович: про російську армію. [23.03.22]
glory to ukraine - слава україні
"лучше таракана нашего е*ите!" частина білоруських військових не буде воювати проти україни
ми всі повинні зупинити росію. світ повинен зупинити війну. я вдячний кожному, 

In [52]:
print(dates[-1])
print(docs[-1])

2022-03-07
ти зовсім сліпий? які нормальні люди? 5-10% від населення? це маргінали, більшість звичайнісінькі фашисти. не треба тут самодурством займатись с приводу мільйонів тільки дивлячись на свого друга.  
ти що не бачиш, що між твоїм другом і середнім росіянином просто ментальна прірва?


In [53]:
#If this outputs anything, there is a problem. Something that should be a string is not...
for i in docs: 
    test = isinstance(i, str)
    if test == False: 
        print (test)

## Training embeddings

In [54]:
import random
random.seed(29)

#Husk at installe sentence_transformers via terminalen
from sentence_transformers import SentenceTransformer

# Prepare embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)


## Fitting the model

In [55]:
#Kør kun hvis chunken nedenfor ikke virker
from umap import UMAP
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric='cosine', random_state = 29)

In [56]:
#Skal virke for at få GPU'en til at køre hurtigere, men der er problemer med at loade cuml pakken

#from cuml.manifold import UMAP
#from cuml.cluster import HDBSCAN

# Create instances of GPU-accelerated UMAP and HDBSCAN
#umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric='cosine', random_state = 29)
#hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

In [57]:


from bertopic import BERTopic

#Kør hvis GPU opsætning IKKE virker
topic_model = BERTopic(language = "multilingual", verbose = True, calculate_probabilities = False, umap_model=umap_model)
#Kør hvis GPU opsætning virker
#topic_model = BERTopic(language = "multilingual", verbose = True, calculate_probabilities = False, umap_model=umap_model, hdbscan_model=hdbscan_model)


topics, probs = topic_model.fit_transform(docs, embeddings)


2022-12-16 12:54:26,717 - BERTopic - Reduced dimensionality
2022-12-16 12:54:26,738 - BERTopic - Clustered reduced embeddings


### Removing stopwords 

In [58]:
topic_model.get_topic_info()[0:11]

,Topic,Count,Name
0,0,475,0_не_слава_україні_на
1,1,87,1_не_на_що_це


In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
#vectorizer_model = CountVectorizer(stop_words="english")

topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

### Investigating the model

In [60]:
topic_model.get_topic_info()[0:11]

,Topic,Count,Name
0,0,475,0_не_слава_на_україні
1,1,87,1_не_на_що_це


### Saving and loading models

In [61]:
#topic_model.save("models/model_medium")

In [62]:
#from bertopic import BERTopic

#topic_model = BERTopic.load("models/model_medium")

## Visualizations

In [63]:
#Tager lang tid at køre, når vi har så mange små topics
#topic_model.visualize_topics()

### Topics over time

### Function for visualizing topics over time

In [64]:
import pandas as pd
from typing import List
import plotly.graph_objects as go
from sklearn.preprocessing import normalize


def visualize_topics_over_time(topic_model,
                               topics_over_time: pd.DataFrame,
                               top_n_topics: int = None,
                               topics: List[int] = None,
                               normalize_frequency: bool = False,
                               custom_labels: bool = False,
                               width: int = 1250,
                               height: int = 450) -> go.Figure:
    """ Visualize topics over time
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics_over_time: The topics you would like to be visualized with the
                          corresponding topic representation
        top_n_topics: To visualize the most frequent topics instead of all
        topics: Select which topics you would like to be visualized
        normalize_frequency: Whether to normalize each topic's frequency individually
        custom_labels: Whether to use custom topic labels that were defined using 
                       `topic_model.set_topic_labels`.
        width: The width of the figure.
        height: The height of the figure.
    Returns:
        A plotly.graph_objects.Figure including all traces
    Examples:
    To visualize the topics over time, simply run:
    ```python
    topics_over_time = topic_model.topics_over_time(docs, timestamps)
    topic_model.visualize_topics_over_time(topics_over_time)
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_topics_over_time(topics_over_time)
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/trump.html"
    style="width:1000px; height: 680px; border: 0px;""></iframe>
    """
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        selected_topics = list(topics)
    elif top_n_topics is not None:
        selected_topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        selected_topics = sorted(freq_df.Topic.to_list())

    # Prepare data
    if topic_model.custom_labels_ is not None and custom_labels:
        topic_names = {key: topic_model.custom_labels_[key + topic_model._outliers] for key, _ in topic_model.topic_labels_.items()}
    else:
        topic_names = {key: value[:40] + "..." if len(value) > 40 else value
                       for key, value in topic_model.topic_labels_.items()}
    topics_over_time["Name"] = topics_over_time.Topic.map(topic_names)
    data = topics_over_time.loc[topics_over_time.Topic.isin(selected_topics), :].sort_values(["Topic", "Timestamp"])

    # Add traces
    fig = go.Figure()
    for index, topic in enumerate(data.Topic.unique()):
        trace_data = data.loc[data.Topic == topic, :]
        topic_name = trace_data.Name.values[0]
        words = trace_data.Words.values
        if normalize_frequency:
            y = normalize(trace_data.Frequency.values.reshape(1, -1))[0]
        else:
            y = trace_data.Frequency
        fig.add_trace(go.Scatter(x=trace_data.Timestamp, y=y,
                                 mode='lines',
                                 marker_color=colors[index % 20],
                                 hoverinfo="text",
                                 name=topic_name,
                                 hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))

    # Styling of the visualization
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)
    fig.update_layout(
        yaxis_title="Normalized Frequency" if normalize_frequency else "Frequency",
        title={
            'text': "<b>Topics over Time",
            'y': .95,
            'x': 0.40,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        template="simple_white",
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
        legend=dict(
            title="<b>Global Topic Representation",
        )
    )
    return fig

In [65]:
# Topics over time
topics_over_time = topic_model.topics_over_time(docs, dates, nr_bins=20)
#topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

visualize_topics_over_time(topic_model, topics_over_time, top_n_topics=20)




# Visualize topics over time with the updated colors
#visualize_topics_over_time(model, topics_over_time)

5it [00:00, 21.54it/s]


## Topics pr class

In [66]:
# Topics per class
#topics_per_class = topic_model.topics_per_class(docs, classes = subreddits)

language = data.language.to_list()
type = data.type.to_list()

topics_per_class = topic_model.topics_per_class(docs, classes = language)




1it [00:00, 20.68it/s]


In [67]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
